In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

In [2]:
# Read all data files of one category.
import os

category = 'plastic'
downsample_steps = {'alive': 2, 'fibro': 5, 'plastic': 5}  # TODO: Rename var.

data_dir = 'data/JulianTrajs/' + category
trajectories = []
filenames = os.listdir(data_dir)    
for filename in filenames:
    trajectory = np.genfromtxt(os.path.join(data_dir, filename))
    for start in range(downsample_steps[category]):
        end = -(downsample_steps[category] - start)
        sliced_trajectory = trajectory[start:end:downsample_steps[category]]
        trajectories.append(sliced_trajectory)
trajectories = np.array(trajectories)
print "Category {}: Found {} files, created {} trajectories".format(category, len(filenames), len(trajectories))

Category plastic: Found 177 files, created 885 trajectories


In [3]:
def to_velocities(x, multiple_trajectories=False):
    return np.diff(x, axis=int(multiple_trajectories)) / 5.
# TODO: Does only work for single trajectory right now, extend to multiple trajectories if needed.
def to_positions(x):
    return np.append(np.zeros((1, 3)), np.cumsum(x * 5., axis=0), axis=0)

In [4]:
trajectories = to_velocities(trajectories, multiple_trajectories=True)

In [5]:
# Normalize trajectories to [-1, 1] for the LSTM network (it outputs values between -1 and 1 by default)
# TODO: Do not use min/max values here but fixed value to generalize to other datasets.
min_value = np.min(trajectories)
max_value = np.max(trajectories)
abs_max_value = max(abs(min_value), abs(max_value))
print abs_max_value
def normalize(x):
    return x / abs_max_value
    #return np.interp(x, [min_value, max_value], [-1., 1.])
def denormalize(x):
    return x * abs_max_value
    #return np.interp(x, [-1., 1.], [min_value, max_value])
    
normalize(2)

10.8507712


0.1843186961678816

In [6]:
trajectories = normalize(trajectories)

In [7]:
batch_size = len(trajectories)
num_timesteps = len(trajectories[0])
#num_timesteps_per_epoch = 50


In [8]:
# Set up the network.
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM

model = Sequential()

# Feed in one time step at a time.
model.add(LSTM(batch_input_shape=(batch_size, 1, 3), output_dim=10, return_sequences=False, stateful=True))
# model.add(LSTM(batch_input_shape=(batch_size, 1, 3), output_dim=100, return_sequences=True, stateful=True))

#model.add(Dropout(0.3))
#model.add(LSTM(output_dim=100, return_sequences=True, stateful=True))
#model.add(LSTM(output_dim=100, return_sequences=True, stateful=True))


#model.add(LSTM(output_dim=100, return_sequences=True, stateful=True))
#model.add(LSTM(output_dim=100, return_sequences=False, stateful=True))
model.add(Dense(output_dim=3))

model.compile(loss='mean_squared_error', optimizer='rmsprop')
model.reset_states()  # TODO: Is this necessary?

Using Theano backend.
Using gpu device 0: GeForce 840M (CNMeM is disabled, CuDNN not available)
D:\Python\27_32bit\lib\site-packages\theano\tensor\signal\downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")


In [9]:
losses = []

In [10]:
num_epochs = 100
for epoch in range(1, num_epochs+1):
    print 'Epoch', epoch,
    losses_epoch = []
    for i in range(num_timesteps - 1):
        results = model.train_on_batch(trajectories[:, i:i+1], trajectories[:, i+1])
        losses_epoch.append(results[0])
    mean_loss = np.mean(losses_epoch)
    print '- loss:', mean_loss
    losses.append(mean_loss)
    model.reset_states()

Epoch 1 - loss: 0.00242808
Epoch 2 - loss: 0.0022587
Epoch 3 - loss: 0.00216513
Epoch 4 - loss: 0.00210526
Epoch 5 - loss: 0.00206357
Epoch 6 - loss: 0.00203296
Epoch 7 - loss: 0.00200966
Epoch 8 - loss: 0.00199141
Epoch 9 - loss: 0.00197674
Epoch 10 - loss: 0.00196469
Epoch 11 - loss: 0.00195457
Epoch 12 - loss: 0.00194592
Epoch 13 - loss: 0.0019384
Epoch 14 - loss: 0.00193178
Epoch 15 - loss: 0.00192589
Epoch 16 - loss: 0.00192059
Epoch 17 - loss: 0.00191579
Epoch 18 - loss: 0.00191143
Epoch 19 - loss: 0.00190744
Epoch 20 - loss: 0.00190378
Epoch 21 - loss: 0.00190041
Epoch 22 - loss: 0.0018973
Epoch 23 - loss: 0.00189443
Epoch 24 - loss: 0.00189176
Epoch 25 - loss: 0.00188929
Epoch 26 - loss: 0.001887
Epoch 27 - loss: 0.00188487
Epoch 28 - loss: 0.00188288
Epoch 29 - loss: 0.00188103
Epoch 30 - loss: 0.00187931
Epoch 31 - loss: 0.00187769
Epoch 32 - loss: 0.00187619
Epoch 33 - loss: 0.00187478
Epoch 34 - loss: 0.00187347
Epoch 35 - loss: 0.00187224
Epoch 36 - loss: 0.00187108
Epoch 

In [ ]:
# TODO: Evaluate if this has a positive effect on final accuracy and/or learning speed.
# Split up trajectories in smaller parts and reset states in between.
num_epochs = 1000
num_steps_per_mini_batch = 50
for epoch in range(1, num_epochs+1):
    print 'Epoch', epoch,
    losses_epoch = []
    for start in range(0, num_timesteps - 1 - num_steps_per_mini_batch, num_steps_per_mini_batch):  # TODO: Shuffle the list created by range and see if this has an effect.
        for i in range(start, start + num_steps_per_mini_batch):
            results = model.train_on_batch(trajectories[:, i:i+1], trajectories[:, i+1])
            losses_epoch.append(results[0])
        model.reset_states()
    mean_loss = np.mean(losses_epoch)
    print '- loss:', mean_loss
    losses.append(mean_loss)
    model.reset_states()

In [ ]:
plt.plot(losses)

In [ ]:
# Predict all steps using the training data as input (i. e. not the predictions!).
model.reset_states()
predicted = []
predicted.append(trajectories[:, 0])  # Take first timestep from training data so predicted and expected have the same dimensions.
for i in range(num_timesteps - 1):
    predicted.append(model.predict_on_batch(trajectories[:, i:i+1])[0])
predicted = np.array(predicted)
predicted = predicted.transpose((1, 0, 2))
expected = trajectories

In [ ]:
# Plot single trajectory from positions.
i_traj = 15
pr = denormalize(predicted[i_traj])
plt.plot(pr[:, 0], pr[:, 1], label='Predicted')
exp = denormalize(expected[i_traj])
plt.plot(exp[:, 0], exp[:, 1], label='Expected')
lim = abs(max(np.min(pr), np.max(pr), np.min(exp), np.max(exp), key=abs)) + 20
plt.xlim(-lim, lim)
plt.ylim(-lim, lim)
plt.legend()

In [ ]:
# Plot single trajectory from velocities.
i_traj = 10
pr = to_positions(denormalize(predicted[i_traj]))
plt.plot(pr[:, 0], pr[:, 1], label='Predicted')
exp = to_positions(denormalize(expected[i_traj]))
plt.plot(exp[:, 0], exp[:, 1], label='Expected')
lim = abs(max(np.min(pr), np.max(pr), np.min(exp), np.max(exp), key=abs)) + 20
plt.xlim(-lim, lim)
plt.ylim(-lim, lim)
plt.legend()
#plt.plot(traj)

# TODO: Due to training on velocities, predicted trajectories are often scaled versions of expected trajectories.

In [ ]:
len_seed = 50  # TODO: Play around with seed size.
predicted = np.zeros((batch_size, num_timesteps, 3))
predicted[:, :len_seed] = trajectories[:, :len_seed]
model.reset_states()

log_traj = 0
    
for i in range(num_timesteps - 1):
    model_input = predicted[:, i:i+1]
    model_output = model.predict_on_batch(model_input)[0]
    model_output += 0.25 * (np.random.rand(batch_size, 3) - 0.5)  # velocities: 0.25
    
    print 'Tr {}, step {}: In: {} - Out: {}'.format(log_traj, i, model_input[log_traj], model_output[log_traj]), 
    
    if i+1 >= len_seed:
        predicted[:, i+1] = model_output
        print '--> Storing at {}'.format(i+1)
    else:
        print '--> Seed'

expected = trajectories

# TODO: Use end of the trajectory as seed and predict how it would continue.
# PROBLEM: LSTM settles into equilibrium after a while, where it always outputs the same velocities.

In [ ]:
# Plot multiple trajectories from positions. 
i_traj = 0
for i_traj in range(0, 200, 5):
    pr = denormalize(predicted[i_traj])
    plt.plot(pr[:, 0], pr[:, 1], label='Predicted')
lim = 300#abs(max(np.min(pr), np.max(pr), np.min(exp), np.max(exp), key=abs)) + 20
plt.xlim(-lim, lim)
plt.ylim(-lim, lim)

In [ ]:
# Plot single trajectory from velocities.
i_traj = 20
pr = to_positions(denormalize(predicted[i_traj]))
plt.plot(pr[:, 0], pr[:, 1], label='Predicted')
exp = to_positions(denormalize(expected[i_traj]))
plt.plot(exp[:, 0], exp[:, 1], label='Expected')
lim = abs(max(np.min(pr), np.max(pr), np.min(exp), np.max(exp), key=abs)) + 20
plt.xlim(-lim, lim)
plt.ylim(-lim, lim)
plt.legend()

In [ ]:
# Plot multiple trajectories from velocities.
for i_traj in range(0, 200, 5):
    pr = to_positions(denormalize(predicted[i_traj]))
    plt.plot(pr[:, 0], pr[:, 1], label='Predicted')
lim = 400#abs(max(np.min(pr), np.max(pr), np.min(exp), np.max(exp), key=abs)) + 20
plt.xlim(-lim, lim)
plt.ylim(-lim, lim)

In [ ]:
# Make completely random trajectories.
# TODO: Make random numbers at start of prediction, then feed them in one at a time. Thus, I can plot both the predicted trajectories and the "random part" of those trajectories.
random_trajectories = 0.2 * (np.random.rand(batch_size, num_timesteps, 3) - 0.5)
for i_traj in range(0, 200, 5):
    pr = to_positions(denormalize(random_trajectories[i_traj]))
    plt.plot(pr[:, 0], pr[:, 1], label='Random')
lim = 50#abs(max(np.min(pr), np.max(pr), np.min(exp), np.max(exp), key=abs)) + 20
plt.xlim(-lim, lim)
plt.ylim(-lim, lim)

In [ ]:
# Predict with random values as seed.
len_seed = 1
predicted = np.zeros((batch_size, num_timesteps, 3))
# Disable this line to predict on 0-inputs (due to the randomness, this also creates different trajectories).
#predicted[:, :len_seed] = 2. * (np.random.rand(batch_size, len_seed, 3) - 0.5)
model.reset_states()

log_traj = 0
    
for i in range(num_timesteps - 1):
    model_input = predicted[:, i:i+1]
    model_output = model.predict_on_batch(model_input)[0]    
    model_output += 0.25 * (np.random.rand(batch_size, 3) - 0.5)
    
    print 'Tr {}, step {}: In: {} - Out: {}'.format(log_traj, i, model_input[log_traj], model_output[log_traj]), 
    
    if i+1 >= len_seed:
        predicted[:, i+1] = model_output
        print '--> Storing at {}'.format(i+1)
    else:
        print '--> Seed'

In [ ]:
# Plot multiple trajectories from velocities.
for i_traj in range(0, 200, 5):
    pr = to_positions(denormalize(predicted[i_traj]))
    plt.plot(pr[:, 0], pr[:, 1], label='Predicted')
lim = 300#abs(max(np.min(pr), np.max(pr), np.min(exp), np.max(exp), key=abs)) + 20
plt.xlim(-lim, lim)
plt.ylim(-lim, lim)